In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.5e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(1)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t> start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, r, _, _, c = env.transition(state, ac, para)  
            s0 += r
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2500000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2500000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2500000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0, Budget: 2500000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

 Time: 11 | S: 25437715.0, E: 645.4, A: 940.2, I: 2162.9, D: 5.4, R: 54531.1, Budget: 2373679.03, reward:-13447.43
 Time: 11 | S: 25437841.0, E: 618.0, A: 940.2, I: 2162.9, D: 5.4, R: 54519.0, Budget: 2373679.03, reward:-13447.43
 Time: 11 | new_e:314.6, cum_e:3012.5, new_i:379.6, cum_i:3037.9
[[0.2788997  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997
  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997
  0.2788997  0.2788997  0.2788997  0.2788997 ]
 [0.59610213 0.59610213 0.59610213 0.59610213 0.59610213 0.59610213
  0.59610213 0.59610213 0.59610213 0.59610213 0.59610213 0.59610213
  0.59610213 0.59610213 0.59610213 0.59610213]
 [0.97678602 0.97678602 0.97678602 0.97678602 0.97678602 0.97678602
  0.97678602 0.97678602 0.97678602 0.97678602 0.97678602 0.97678602
  0.97678602 0.97678602 0.97678602 0.97678602]
 [0.99795483 0.99795483 0.99795483 0.99795483 0.99795483 0.99795483
  0.99795483 0.99795483 0.99795483 0.99795483 0.99795483 0.99795483
  0.99795483 0.9979548

 Time: 18 | S: 25256140.3, E: 225.4, A: 1127.7, I: 2607.3, D: 11.7, R: 235887.4, Budget: 2233938.25, reward:-19921.49
 Time: 18 | S: 25256315.0, E: 219.0, A: 1127.7, I: 2607.3, D: 11.7, R: 235882.0, Budget: 2233938.25, reward:-19921.49
 Time: 18 | new_e:276.4, cum_e:4423.7, new_i:329.3, cum_i:5134.6
[[0.018867   0.018867   0.018867   0.018867   0.018867   0.018867
  0.018867   0.018867   0.018867   0.018867   0.018867   0.018867
  0.018867   0.018867   0.018867   0.018867  ]
 [0.81068605 0.81068605 0.81068605 0.81068605 0.81068605 0.81068605
  0.81068605 0.81068605 0.81068605 0.81068605 0.81068605 0.81068605
  0.81068605 0.81068605 0.81068605 0.81068605]
 [0.84719361 0.84719361 0.84719361 0.84719361 0.84719361 0.84719361
  0.84719361 0.84719361 0.84719361 0.84719361 0.84719361 0.84719361
  0.84719361 0.84719361 0.84719361 0.84719361]
 [0.62985536 0.62985536 0.62985536 0.62985536 0.62985536 0.62985536
  0.62985536 0.62985536 0.62985536 0.62985536 0.62985536 0.62985536
  0.62985536 0.629

 Time: 25 | S: 25108615.9, E: 594.3, A: 822.1, I: 2872.1, D: 19.2, R: 383076.5, Budget: 2051779.56, reward:-27144.41
 Time: 25 | S: 25108797.0, E: 602.0, A: 822.1, I: 2872.1, D: 19.2, R: 383066.0, Budget: 2051779.56, reward:-27144.41
 Time: 25 | new_e:300.2, cum_e:5986.0, new_i:351.9, cum_i:7218.7
[[0.94786249 0.94786249 0.94786249 0.94786249 0.94786249 0.94786249
  0.94786249 0.94786249 0.94786249 0.94786249 0.94786249 0.94786249
  0.94786249 0.94786249 0.94786249 0.94786249]
 [0.80647933 0.80647933 0.80647933 0.80647933 0.80647933 0.80647933
  0.80647933 0.80647933 0.80647933 0.80647933 0.80647933 0.80647933
  0.80647933 0.80647933 0.80647933 0.80647933]
 [0.94529084 0.94529084 0.94529084 0.94529084 0.94529084 0.94529084
  0.94529084 0.94529084 0.94529084 0.94529084 0.94529084 0.94529084
  0.94529084 0.94529084 0.94529084 0.94529084]
 [0.98578841 0.98578841 0.98578841 0.98578841 0.98578841 0.98578841
  0.98578841 0.98578841 0.98578841 0.98578841 0.98578841 0.98578841
  0.98578841 0.9

 Time: 32 | S: 24989066.4, E: 763.9, A: 724.4, I: 3157.1, D: 27.6, R: 502260.6, Budget: 1938381.95, reward:-35145.40
 Time: 32 | S: 24989232.0, E: 792.0, A: 724.4, I: 3157.1, D: 27.6, R: 502272.0, Budget: 1938381.95, reward:-35145.40
 Time: 32 | new_e:366.1, cum_e:7704.8, new_i:403.5, cum_i:9482.5
[[0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276]
 [0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977]
 [0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024]
 [0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.9

...

 Time: 60 | S: 24542674.6, E: 107.8, A: 829.2, I: 2438.2, D: 59.0, R: 949891.2, Budget: 1385841.91, reward:-57199.36
 Time: 60 | S: 24542559.0, E: 108.0, A: 829.2, I: 2438.2, D: 59.0, R: 949971.0, Budget: 1385841.91, reward:-57199.36
 Time: 60 | new_e:75.0, cum_e:12378.3, new_i:150.4, cum_i:16412.0
[[0.04167526 0.04167526 0.04167526 0.04167526 0.04167526 0.04167526
  0.04167526 0.04167526 0.04167526 0.04167526 0.04167526 0.04167526
  0.04167526 0.04167526 0.04167526 0.04167526]
 [0.97695401 0.97695401 0.97695401 0.97695401 0.97695401 0.97695401
  0.97695401 0.97695401 0.97695401 0.97695401 0.97695401 0.97695401
  0.97695401 0.97695401 0.97695401 0.97695401]
 [0.97715547 0.97715547 0.97715547 0.97715547 0.97715547 0.97715547
  0.97715547 0.97715547 0.97715547 0.97715547 0.97715547 0.97715547
  0.97715547 0.97715547 0.97715547 0.97715547]
 [0.92148621 0.92148621 0.92148621 0.92148621 0.92148621 0.92148621
  0.92148621 0.92148621 0.92148621 0.92148621 0.92148621 0.92148621
  0.92148621 0.

 Time: 67 | S: 24422553.3, E: 36.8, A: 423.8, I: 1749.0, D: 64.9, R: 1071172.1, Budget: 1204922.04, reward:-58703.74
 Time: 67 | S: 24422444.0, E: 31.0, A: 423.8, I: 1749.0, D: 64.9, R: 1071242.0, Budget: 1204922.04, reward:-58703.74
 Time: 67 | new_e:32.7, cum_e:12630.7, new_i:71.2, cum_i:17070.6
[[0.05491752 0.05491752 0.05491752 0.05491752 0.05491752 0.05491752
  0.05491752 0.05491752 0.05491752 0.05491752 0.05491752 0.05491752
  0.05491752 0.05491752 0.05491752 0.05491752]
 [0.35696556 0.35696556 0.35696556 0.35696556 0.35696556 0.35696556
  0.35696556 0.35696556 0.35696556 0.35696556 0.35696556 0.35696556
  0.35696556 0.35696556 0.35696556 0.35696556]
 [0.97815331 0.97815331 0.97815331 0.97815331 0.97815331 0.97815331
  0.97815331 0.97815331 0.97815331 0.97815331 0.97815331 0.97815331
  0.97815331 0.97815331 0.97815331 0.97815331]
 [0.92487546 0.92487546 0.92487546 0.92487546 0.92487546 0.92487546
  0.92487546 0.92487546 0.92487546 0.92487546 0.92487546 0.92487546
  0.92487546 0.9

 Time: 74 | S: 24285126.7, E: 12.9, A: 205.4, I: 1118.0, D: 68.9, R: 1209468.1, Budget: 962521.90, reward:-59342.19
 Time: 74 | S: 24285037.0, E: 16.0, A: 205.4, I: 1118.0, D: 68.9, R: 1209541.0, Budget: 962521.90, reward:-59342.19
 Time: 74 | new_e:8.1, cum_e:12736.8, new_i:28.8, cum_i:17377.3
[[0.85167474 0.85167474 0.85167474 0.85167474 0.85167474 0.85167474
  0.85167474 0.85167474 0.85167474 0.85167474 0.85167474 0.85167474
  0.85167474 0.85167474 0.85167474 0.85167474]
 [0.88450176 0.88450176 0.88450176 0.88450176 0.88450176 0.88450176
  0.88450176 0.88450176 0.88450176 0.88450176 0.88450176 0.88450176
  0.88450176 0.88450176 0.88450176 0.88450176]
 [0.98018757 0.98018757 0.98018757 0.98018757 0.98018757 0.98018757
  0.98018757 0.98018757 0.98018757 0.98018757 0.98018757 0.98018757
  0.98018757 0.98018757 0.98018757 0.98018757]
 [0.99500378 0.99500378 0.99500378 0.99500378 0.99500378 0.99500378
  0.99500378 0.99500378 0.99500378 0.99500378 0.99500378 0.99500378
  0.99500378 0.9950

 Time: 81 | S: 23979896.1, E: 4.2, A: 80.7, I: 566.2, D: 70.1, R: 1515382.7, Budget: 717318.54, reward:-59509.76
 Time: 81 | S: 23979790.0, E: 0.0, A: 80.7, I: 566.2, D: 70.1, R: 1515452.0, Budget: 717318.54, reward:-59509.76
 Time: 81 | new_e:2.1, cum_e:12760.7, new_i:10.5, cum_i:17493.6
[[0.20783556 0.20783556 0.20783556 0.20783556 0.20783556 0.20783556
  0.20783556 0.20783556 0.20783556 0.20783556 0.20783556 0.20783556
  0.20783556 0.20783556 0.20783556 0.20783556]
 [0.88494578 0.88494578 0.88494578 0.88494578 0.88494578 0.88494578
  0.88494578 0.88494578 0.88494578 0.88494578 0.88494578 0.88494578
  0.88494578 0.88494578 0.88494578 0.88494578]
 [0.99547294 0.99547294 0.99547294 0.99547294 0.99547294 0.99547294
  0.99547294 0.99547294 0.99547294 0.99547294 0.99547294 0.99547294
  0.99547294 0.99547294 0.99547294 0.99547294]
 [0.99773228 0.99773228 0.99773228 0.99773228 0.99773228 0.99773228
  0.99773228 0.99773228 0.99773228 0.99773228 0.99773228 0.99773228
  0.99773228 0.99773228 0

 Time: 88 | S: 23793266.9, E: 9.1, A: 31.2, I: 271.2, D: 70.6, R: 1702350.9, Budget: 567733.00, reward:-59633.56
 Time: 88 | S: 23793191.0, E: 0.0, A: 31.2, I: 271.2, D: 70.6, R: 1702415.0, Budget: 567733.00, reward:-59633.56
 Time: 88 | new_e:4.3, cum_e:12783.6, new_i:7.2, cum_i:17549.9
[[0.53379645 0.53379645 0.53379645 0.53379645 0.53379645 0.53379645
  0.53379645 0.53379645 0.53379645 0.53379645 0.53379645 0.53379645
  0.53379645 0.53379645 0.53379645 0.53379645]
 [0.92538568 0.92538568 0.92538568 0.92538568 0.92538568 0.92538568
  0.92538568 0.92538568 0.92538568 0.92538568 0.92538568 0.92538568
  0.92538568 0.92538568 0.92538568 0.92538568]
 [0.99336831 0.99336831 0.99336831 0.99336831 0.99336831 0.99336831
  0.99336831 0.99336831 0.99336831 0.99336831 0.99336831 0.99336831
  0.99336831 0.99336831 0.99336831 0.99336831]
 [0.99336566 0.99336566 0.99336566 0.99336566 0.99336566 0.99336566
  0.99336566 0.99336566 0.99336566 0.99336566 0.99336566 0.99336566
  0.99336566 0.99336566 0.

In [6]:
(time.time() - tic) / ( T - start_time + 1)

82.2182871874641